# Credentials and initialize conneciton

Follow https://developers.google.com/earth-engine/reference/Quickstart#before-you-begin to get a secret key

In [1]:
import json

KEY = 'my-secret-key.json'
with open(KEY, 'r') as f:
    data = json.load(f)
    SERVICE_ACCOUNT = data['client_email']
    PROJECT = data['project_id']

In [2]:
import ee

ee_creds = ee.ServiceAccountCredentials(SERVICE_ACCOUNT, KEY)
ee.Initialize(ee_creds)

# NDVI around coordinate

In [4]:
import geemap

# --- Add NDVI and date index bands before masking ---
def add_ndvi_and_index_before_mask(image, index):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    index_band = ee.Image.constant(index).rename('index').toFloat()
    return image.addBands([ndvi, index_band])

# --- Mask Sentinel-2 bad data ---
def mask_s2_bad_data(image):
    scl = image.select('SCL')
    bad_classes = [0, 1, 2, 3, 8, 9, 10]
    mask = scl.remap(bad_classes, [0]*len(bad_classes), 1).eq(1)
    return image.updateMask(mask)

# --- Pull area function ---
def pull_area(geom, time_start, time_end):
    # Sentinel-2 image collection
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40))
        .filterBounds(geom)
        .filterDate(time_start, time_end)
        .sort('system:time_start', True)
    )

    images_list = collection.toList(collection.size())
    dates = []
    processed_images = []

    # Process each image
    for i in range(images_list.size().getInfo()):
        img = ee.Image(images_list.get(i))
        date_str = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        dates.append(date_str)
        img_with_bands = add_ndvi_and_index_before_mask(img, i)
        img_masked = mask_s2_bad_data(img_with_bands)
        processed_images.append(img_masked)

    # Rebuild collection
    indexed_collection = ee.ImageCollection(processed_images)
    filled = indexed_collection.qualityMosaic('index').clip(geom)

    # Extract bands
    ndvi = filled.select('NDVI')
    index = filled.select('index')
    newest_raw = collection.sort('system:time_start', False).first().clip(geom)
    rgb_raw = newest_raw.select(['B4', 'B3', 'B2']).divide(10000)
    rgb_filled = filled.select(['B4', 'B3', 'B2']).divide(10000)
    swi_filled = filled.normalizedDifference(['B8', 'B11']).rename('SWI').gt(0.2)
    ndvi_filtered = ndvi.updateMask(ndvi.gt(0.3))

    # Dynamic World grass mosaic
    dw_collection = (
        ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
        .filterBounds(geom)
        .filterDate(time_start, time_end)
    )

    dw_image = dw_collection.qualityMosaic('grass').clip(geom)
    dw_grass_mask = dw_image.select('grass').gt(0.3)

    # Create vector outline
    dw_outline = dw_grass_mask.reduceToVectors(
        geometry=geom,
        scale=50,
        geometryType='polygon',
        eightConnected=False,
        labelProperty='grass',
        reducer=ee.Reducer.countEvery()
    )

    # Simplify grass geometry 
    dw_outline = (
        dw_outline
        .filter(ee.Filter.gt('grass', 0))
        .map(lambda f: f.buffer(30).simplify(30))
        .map(lambda f: f.set('area', f.geometry().area()))
        .filter(ee.Filter.gt('area', 100000))  # ~1 ha
    )

    # Mask NDVI to Dynamic World grass areas
    dw_mask = ee.Image(0).byte().paint(featureCollection=dw_outline, color=1)
    masked_ndvi = ndvi.updateMask(dw_mask)

    # Visualization params
    ndvi_vis = {'min': 0, 'max': 1.0, 'palette': ['red', 'yellow', 'green']}
    index_vis = {'min': 0, 'max': len(dates) - 1, 'palette': ['black', 'white']}
    rgb_vis = {'min': 0.0, 'max': 0.3}
    dw_params = {'min': 0, 'max': 1, 'palette': ['white', 'green']}
    masked_ndvi_vis = {'min': 0, 'max': 1.0, 'palette': ['red', 'yellow', 'green']}
    swi_vis = {'min': 0, 'max': 1, 'palette': ['white', 'blue']}

    # Get the center of the bounding box as [lat, lon]
    coords = geom.bounds().coordinates().getInfo()[0]
    min_lon = min([c[0] for c in coords])
    max_lon = max([c[0] for c in coords])
    min_lat = min([c[1] for c in coords])
    max_lat = max([c[1] for c in coords])
    bbox_center = [(min_lat + max_lat) / 2, (min_lon + max_lon) / 2]

    # Create and populate map
    Map = geemap.Map()
    Map.centerObject(geom, 12)
    Map.addLayer(ndvi, ndvi_vis, 'Cloud-Filled NDVI')
    Map.addLayer(index, index_vis, 'Image Recency Index (black=newest)')
    Map.addLayer(rgb_raw, rgb_vis, 'RGB (Raw)')
    Map.addLayer(rgb_filled, rgb_vis, 'RGB Composite (Via SCL)')
    Map.addLayer(dw_image.select('grass'), dw_params, 'Dynamic World (Grass)')
    Map.addLayer(dw_outline, {'color': 'blue'}, 'Dynamic World (Grass Outline)')
    Map.addLayer(masked_ndvi, masked_ndvi_vis, 'NDVI (Masked to Grass)')
    Map.addLayer(swi_filled, swi_vis, 'SWI')
    Map.addLayer(ndvi_filtered, ndvi_vis, 'NDVI (Filtered > 0.3)')

    # Print mapping
    print("Index to Date mapping:")
    for idx, d in enumerate(dates):
        print(f"Index {idx}: {d}")

    return Map, swi_filled

# Example usage
geom = ee.Geometry.BBox(36.1767461932, 4.1184180005, 36.4138149184, 4.3365835632)
time_start = '2025-05-14'
time_end = '2025-06-14'
Map, swi_filled = pull_area(geom, time_start, time_end)



Index to Date mapping:
Index 0: 2025-05-15
Index 1: 2025-05-20
Index 2: 2025-05-20
Index 3: 2025-05-25
Index 4: 2025-05-25
Index 5: 2025-06-01
Index 6: 2025-06-01
Index 7: 2025-06-04
Index 8: 2025-06-04
Index 9: 2025-06-09
Index 10: 2025-06-09
Index 11: 2025-06-11
Index 12: 2025-06-11


In [ ]:
swi_outline = swi_filled.reduceToVectors(
        geometry=geom,
        scale=50,
        geometryType='polygon',
        eightConnected=True,
        labelProperty='SWI',
        reducer=ee.Reducer.countEvery()
    )

# Simplify SWI geometry
swi_outline = (
    swi_outline
    .filter(ee.Filter.gt('SWI', 0))
    .map(lambda f: f.buffer(30).simplify(30))
    .map(lambda f: f.set('area', f.geometry().area()))
)

Map.addLayer(swi_outline, {'color': 'orange'}, 'SWI (Outline)')

In [ ]:
Map

In [5]:
import geemap

AOI = ee.Geometry.BBox(36.1767461932, 4.1184180005, 36.4138149184, 4.3365835632)
START_DATE = '2025-05-14'
END_DATE = '2025-06-14'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 30
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)

def apply_cld_shdw_mask(img):
    not_cld_shdw = img.select('cloudmask').Not()
    return img.updateMask(not_cld_shdw)          # keep all bands, we’ll add more soon

def add_timestamp_band(img):
    ts = ee.Image.constant(ee.Number(img.get('system:time_start'))) \
                  .rename('timestamp') \
                  .toInt64() \
                  .updateMask(img.select('B8').mask())  # use a single-band mask
    return img.addBands(ts)


latest_clear = (s2_sr_cld_col
                .map(add_cld_shdw_mask)
                .map(apply_cld_shdw_mask)
                .map(add_timestamp_band)
                .qualityMosaic('timestamp')
                .clip(AOI)
                .select(['B4', 'B3', 'B2', 'timestamp'])
                .divide(10000)
                )

latest_clear_rgb = latest_clear.select(['B4', 'B3', 'B2'])

latest_clear_timestamp = latest_clear.select('timestamp')

                             

# Map = geemap.Map()
# Map.centerObject(AOI, 15)

most_recent_raw = ee.Image(
    s2_sr_cld_col.sort('system:time_start', False).first()).clip(AOI).select(['B4', 'B3', 'B2']).divide(10000)

rgb_vis = {'min': 0.0, 'max': 0.3}
latest_clear_timestamp_vis = {
    'min': ee.Date(START_DATE).millis().getInfo(),
    'max': ee.Date(END_DATE).millis().getInfo(),
    'palette': ['red', 'yellow', 'green']
}

# True‑colour RGB
Map.addLayer(latest_clear_rgb,
           rgb_vis,
           'Latest clear pixel mosaic')

# Map.addLayer(latest_clear_timestamp, latest_clear_timestamp_vis, 'Pixel date (newer → green)')

Map  # Show the map

Map(center=[4.227498670954776, 36.295280555799295], controls=(WidgetControl(options=['position', 'transparent_…